In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypertension"
cohort = "GSE71994"

# Input paths
in_trait_dir = "../../input/GEO/Hypertension"
in_cohort_dir = "../../input/GEO/Hypertension/GSE71994"

# Output paths
out_data_file = "../../output/preprocess/Hypertension/GSE71994.csv"
out_gene_data_file = "../../output/preprocess/Hypertension/gene_data/GSE71994.csv"
out_clinical_data_file = "../../output/preprocess/Hypertension/clinical_data/GSE71994.csv"
json_path = "../../output/preprocess/Hypertension/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"PBMC expression signatures of controlled and uncontrolled hypertensives"
!Series_summary	"There is high variability in responses to drug therapy for hypertension. Gene expression may help enlighten molecular mechanisms of genome control underlying pathophysiology."
!Series_summary	"This study identifies and characterizes, in replicate samples, a gene expression signature that classifies individuals into controlled and uncontrolled hypertensives."
!Series_overall_design	"20 hypertensive patients (58±13 yrs) under a standardized antihypertensive medication were examined twice, five months apart with measures of 24-hour ambulatory blood pressure (BP) monitoring and genome-wide gene expression analysis of peripheral blood mononuclear cells (PBMCs)."
Sample Characteristics Dictionary:
{0: ['tissue: peripheral blood mononuclear cells'], 1: ['gender: female', 'gender: male'], 2: ['race: white', 'race: brown', 'race: black'], 3: ['age: 32', 'age: 36', 'ag

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the dataset description, this study involves genome-wide gene expression analysis
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For hypertension, we can use systolic blood pressure as an indicator
# Row 6 contains "sistolic blood pressure in a sample" which can be used to determine hypertension
trait_row = 6

# Age is available in row 3
age_row = 3

# Gender is available in row 1
gender_row = 1

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert systolic blood pressure to hypertension binary value.
    According to standard guidelines, systolic BP ≥ 140 mmHg indicates hypertension."""
    try:
        if ":" in value:
            # Extract value after colon
            bp_str = value.split(":", 1)[1].strip()
            bp = int(bp_str)
            # Use 140 as cutoff for hypertension (1=hypertension, 0=normal)
            return 1 if bp >= 140 else 0
        return None
    except (ValueError, TypeError):
        return None

def convert_age(value):
    """Convert age value to numeric."""
    try:
        if ":" in value:
            age_str = value.split(":", 1)[1].strip()
            return int(age_str)
        return None
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)."""
    if ":" in value:
        gender = value.split(":", 1)[1].strip().lower()
        if "female" in gender:
            return 0
        elif "male" in gender:
            return 1
    return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM1849376': [0.0, 32.0, 0.0], 'GSM1849377': [1.0, 36.0, 0.0], 'GSM1849378': [0.0, 38.0, 1.0], 'GSM1849379': [0.0, 48.0, 1.0], 'GSM1849380': [1.0, 52.0, 0.0], 'GSM1849381': [1.0, 53.0, 1.0], 'GSM1849382': [1.0, 55.0, 0.0], 'GSM1849383': [0.0, 58.0, 1.0], 'GSM1849384': [1.0, 59.0, 0.0], 'GSM1849385': [1.0, 59.0, 0.0], 'GSM1849386': [0.0, 59.0, 1.0], 'GSM1849387': [0.0, 61.0, 0.0], 'GSM1849388': [0.0, 62.0, 1.0], 'GSM1849389': [0.0, 64.0, 0.0], 'GSM1849390': [0.0, 66.0, 0.0], 'GSM1849391': [1.0, 68.0, 1.0], 'GSM1849392': [0.0, 69.0, 0.0], 'GSM1849393': [0.0, 71.0, 0.0], 'GSM1849394': [1.0, 73.0, 0.0], 'GSM1849395': [0.0, 79.0, 0.0], 'GSM1849396': [0.0, 38.0, 1.0], 'GSM1849397': [0.0, 71.0, 0.0], 'GSM1849398': [0.0, 59.0, 1.0], 'GSM1849399': [0.0, 69.0, 0.0], 'GSM1849400': [0.0, 32.0, 0.0], 'GSM1849401': [0.0, 64.0, 0.0], 'GSM1849402': [0.0, 58.0, 1.0], 'GSM1849403': [0.0, 66.0, 0.0], 'GSM1849404': [0.0, 48.0, 1.0], 'GSM1849405': [0.0, 61.0, 0.0], 

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['7896746', '7896756', '7896759', '7896761', '7896779', '7896798',
       '7896817', '7896822', '7896859', '7896861', '7896863', '7896865',
       '7896878', '7896882', '7896908', '7896917', '7896921', '7896929',
       '7896937', '7896952'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These appear to be probe IDs from a microarray platform, not human gene symbols
# They are numeric identifiers that need to be mapped to actual gene symbols
# This format is common in Affymetrix or similar microarray platforms

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // ---

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the key columns in the gene annotation dataframe that correspond to 
#    probe IDs and gene symbols.
# Looking at the gene_annotation preview, we can see:
# - 'ID' contains the probe identifiers (matching the gene expression data's index)
# - 'gene_assignment' contains gene symbol information

# 2. Create a gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the first few rows of the mapped gene expression data
print("Gene expression data after mapping:")
print(gene_data.shape)
print(gene_data.index[:10])  # Show the first 10 gene symbols


Gene expression data after mapping:
(117447, 40)
Index(['A-', 'A-3-', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Re-extract clinical data properly from the matrix file
# Get the sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Extract clinical features with controlled vs uncontrolled hypertension based on systolic blood pressure
# Row 6 contains systolic blood pressure values for the first measurement
# We'll use this to classify patients as controlled (<140 mmHg) or uncontrolled (>=140 mmHg)
def convert_hypertension_control(value):
    """Convert systolic blood pressure to controlled/uncontrolled hypertension.
    Using 140 mmHg as the threshold: <140 = controlled (0), >=140 = uncontrolled (1)"""
    try:
        if ":" in value:
            bp_str = value.split(":", 1)[1].strip()
            bp = int(bp_str)
            # 0 = controlled, 1 = uncontrolled
            return 1 if bp >= 140 else 0
        return None
    except (ValueError, TypeError):
        return None

def convert_age(value):
    """Convert age value to numeric."""
    try:
        if ":" in value:
            age_str = value.split(":", 1)[1].strip()
            return int(age_str)
        return None
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)."""
    if ":" in value:
        gender = value.split(":", 1)[1].strip().lower()
        if "female" in gender:
            return 0
        elif "male" in gender:
            return 1
    return None

# Sample characteristics rows have already been identified:
# Row 6: Systolic BP for first measurement
# Row 3: Age
# Row 1: Gender
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=6,  # Row for systolic BP
    convert_trait=convert_hypertension_control,
    age_row=3,    # Row for age
    convert_age=convert_age,
    gender_row=1, # Row for gender
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
note = "Dataset contains PBMC samples from hypertensive patients classified as controlled (<140 mmHg) or uncontrolled (>=140 mmHg)."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=note
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Processed dataset saved to {out_data_file}")
else:
    print("Dataset not usable due to bias in trait distribution. Data not saved.")

Normalized gene data saved to ../../output/preprocess/Hypertension/gene_data/GSE71994.csv
Clinical data saved to ../../output/preprocess/Hypertension/clinical_data/GSE71994.csv
Clinical data preview:
{'GSM1849376': [0.0, 32.0, 0.0], 'GSM1849377': [1.0, 36.0, 0.0], 'GSM1849378': [0.0, 38.0, 1.0], 'GSM1849379': [0.0, 48.0, 1.0], 'GSM1849380': [1.0, 52.0, 0.0], 'GSM1849381': [1.0, 53.0, 1.0], 'GSM1849382': [1.0, 55.0, 0.0], 'GSM1849383': [0.0, 58.0, 1.0], 'GSM1849384': [1.0, 59.0, 0.0], 'GSM1849385': [1.0, 59.0, 0.0], 'GSM1849386': [0.0, 59.0, 1.0], 'GSM1849387': [0.0, 61.0, 0.0], 'GSM1849388': [0.0, 62.0, 1.0], 'GSM1849389': [0.0, 64.0, 0.0], 'GSM1849390': [0.0, 66.0, 0.0], 'GSM1849391': [1.0, 68.0, 1.0], 'GSM1849392': [0.0, 69.0, 0.0], 'GSM1849393': [0.0, 71.0, 0.0], 'GSM1849394': [1.0, 73.0, 0.0], 'GSM1849395': [0.0, 79.0, 0.0], 'GSM1849396': [0.0, 38.0, 1.0], 'GSM1849397': [0.0, 71.0, 0.0], 'GSM1849398': [0.0, 59.0, 1.0], 'GSM1849399': [0.0, 69.0, 0.0], 'GSM1849400': [0.0, 32.0, 0.0],

Data shape after handling missing values: (40, 24224)
For the feature 'Hypertension', the least common label is '1.0' with 16 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Hypertension' in this dataset is fine.

Quartiles for 'Age':
  25%: 52.75
  50% (Median): 59.0
  75%: 66.5
Min: 32.0
Max: 79.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 14 occurrences. This represents 35.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Processed dataset saved to ../../output/preprocess/Hypertension/GSE71994.csv
